In [2]:
import pandas as pd

In [3]:
anime = pd.read_csv('animes.csv')

In [4]:
anime.head()

,uid,title,synopsis,genre,aired,episodes,members,popularity,ranked,score,img_url,link
0,28891,Haikyuu!! Second Season,Following their participation at the Inter-Hig...,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun...","Oct 4, 2015 to Mar 27, 2016",25.0,489888,141,25.0,8.82,https://cdn.myanimelist.net/images/anime/9/766...,https://myanimelist.net/anime/28891/Haikyuu_Se...
1,23273,Shigatsu wa Kimi no Uso,Music accompanies the path of the human metron...,"['Drama', 'Music', 'Romance', 'School', 'Shoun...","Oct 10, 2014 to Mar 20, 2015",22.0,995473,28,24.0,8.83,https://cdn.myanimelist.net/images/anime/3/671...,https://myanimelist.net/anime/23273/Shigatsu_w...
2,34599,Made in Abyss,The Abyss—a gaping chasm stretching down into ...,"['Sci-Fi', 'Adventure', 'Mystery', 'Drama', 'F...","Jul 7, 2017 to Sep 29, 2017",13.0,581663,98,23.0,8.83,https://cdn.myanimelist.net/images/anime/6/867...,https://myanimelist.net/anime/34599/Made_in_Abyss
3,5114,Fullmetal Alchemist: Brotherhood,"""In order for something to be obtained, someth...","['Action', 'Military', 'Adventure', 'Comedy', ...","Apr 5, 2009 to Jul 4, 2010",64.0,1615084,4,1.0,9.23,https://cdn.myanimelist.net/images/anime/1223/...,https://myanimelist.net/anime/5114/Fullmetal_A...
4,31758,Kizumonogatari III: Reiketsu-hen,After helping revive the legendary vampire Kis...,"['Action', 'Mystery', 'Supernatural', 'Vampire']","Jan 6, 2017",1.0,214621,502,22.0,8.83,https://cdn.myanimelist.net/images/anime/3/815...,https://myanimelist.net/anime/31758/Kizumonoga...


In [6]:
import ast
anime['genre'] = anime['genre'].apply(ast.literal_eval)

In [7]:
unique_genre = set(g for genre in anime['genre'] for g in genre)
all_genre = [g for genre in anime['genre'] for g in genre]
unique_genre

{'Action',
 'Adventure',
 'Cars',
 'Comedy',
 'Dementia',
 'Demons',
 'Drama',
 'Ecchi',
 'Fantasy',
 'Game',
 'Harem',
 'Hentai',
 'Historical',
 'Horror',
 'Josei',
 'Kids',
 'Magic',
 'Martial Arts',
 'Mecha',
 'Military',
 'Music',
 'Mystery',
 'Parody',
 'Police',
 'Psychological',
 'Romance',
 'Samurai',
 'School',
 'Sci-Fi',
 'Seinen',
 'Shoujo',
 'Shoujo Ai',
 'Shounen',
 'Shounen Ai',
 'Slice of Life',
 'Space',
 'Sports',
 'Super Power',
 'Supernatural',
 'Thriller',
 'Vampire',
 'Yaoi',
 'Yuri'}

In [8]:
genre_counts = pd.Series(all_genre).value_counts()
genre_counts

Comedy           6461
Action           4215
Fantasy          3466
Adventure        3143
Drama            3004
Sci-Fi           2832
Hentai           2574
Kids             2549
Shounen          2322
Romance          2152
Slice of Life    2056
Music            1993
School           1912
Supernatural     1719
Historical       1218
Mecha            1197
Magic            1174
Seinen            982
Mystery           874
Sports            811
Ecchi             805
Shoujo            788
Super Power       716
Parody            702
Military          678
Demons            605
Space             564
Horror            511
Harem             435
Martial Arts      424
Dementia          419
Psychological     415
Game              395
Police            301
Samurai           238
Vampire           165
Thriller          161
Cars              133
Josei             115
Shounen Ai        113
Shoujo Ai          87
Yuri               79
Yaoi               78
dtype: int64

In [9]:
anime.shape[0]

19311

In [11]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

# Assuming 'df' is your dataframe with missing values
# Identify dependent and independent variables
dependent_variable = 'score'
independent_variables = ['members']

# Split data into two sets
complete_data = anime.dropna(subset=[dependent_variable] + independent_variables)
missing_data = anime[anime[dependent_variable].isnull()]

# Build regression model
X = complete_data[independent_variables]
y = complete_data[dependent_variable]
model = LinearRegression()
model.fit(X, y)

# Predict missing values
missing_data_X = missing_data[independent_variables]
predicted_values = model.predict(missing_data_X)

# Update dataset with predicted values
anime.loc[anime[dependent_variable].isnull(), dependent_variable] = predicted_values

In [12]:
missing_value = anime[['genre','title','members','score']].isnull().any(axis=1)

In [13]:
missing_value

0        False
1        False
2        False
3        False
4        False
         ...  
19306    False
19307    False
19308    False
19309    False
19310    False
Length: 19311, dtype: bool

In [14]:
anime[missing_value]

,uid,title,synopsis,genre,aired,episodes,members,popularity,ranked,score,img_url,link


In [15]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [16]:
# Calculate IDF for genres
vectorizer = TfidfVectorizer(tokenizer=lambda x: x.split())
tfidf_matrix_genre = vectorizer.fit_transform(anime['genre'].apply(lambda x: ' '.join(x)))
#idf_values_genre = dict(zip(vectorizer.get_feature_names_out(), np.log(len(anime) / (1 + vectorizer.idf_))))
#idf_values_genre = {key.capitalize(): value for key, value in idf_values_genre.items()}

# Total number of movies (documents)
total_anime = len(anime)

# Calculate IDF values based on genre_count
idf_values_genre = {genre: np.log(total_anime / (1 + count)) for genre, count in genre_counts.items()}

In [17]:
idf_values_genre

{'Comedy': 1.0947360143362819,
 'Action': 1.5217880704995272,
 'Fantasy': 1.717385215035702,
 'Adventure': 1.8151790071716283,
 'Drama': 1.8603973137514176,
 'Sci-Fi': 1.919338660890209,
 'Hentai': 2.0148253476228892,
 'Kids': 2.024581522568254,
 'Shounen': 2.1178154279503163,
 'Romance': 2.1938126633563626,
 'Slice of Life': 2.2394262710677686,
 'Music': 2.270532210198942,
 'School': 2.3120021912804725,
 'Supernatural': 2.4183505909132266,
 'Historical': 2.762644031239454,
 'Mecha': 2.780021382045331,
 'Magic': 2.799406734142466,
 'Seinen': 2.9778210405735592,
 'Mystery': 3.094206274363111,
 'Sports': 3.1689298205590477,
 'Ecchi': 3.1763464182140972,
 'Shoujo': 3.1976638398748514,
 'Super Power': 3.293354320121105,
 'Parody': 3.3130732689100606,
 'Military': 3.3478090331620294,
 'Demons': 3.461550174651411,
 'Space': 3.5316044295742843,
 'Horror': 3.630105535681218,
 'Harem': 3.7907879173716914,
 'Martial Arts': 3.816340991796309,
 'Dementia': 3.8281754494433113,
 'Psychological': 3.8

In [18]:
# Calculate Genre Score for each movie
anime['genre_score'] = anime['genre'].apply(lambda x: sum(idf_values_genre[genre] for genre in x) / len(x) if len(x) > 0 else 0)

In [19]:
anime['genre_score']

0        2.110776
1        2.150912
2        2.081301
3        2.034315
4        2.947697
           ...   
19306    1.907165
19307    2.361952
19308    3.047554
19309    2.160279
19310    2.687834
Name: genre_score, Length: 19311, dtype: float64

In [20]:
# Normalize Rating and Popularity scores (assuming these are numerical columns)
anime['rating_score'] = anime['score'] / anime['score'].max()
anime['popularity_score'] = anime['members'] / anime['members'].max()

# Combine features with specified weights
anime['combined_score'] = (anime['genre_score'] * 0.6) + (anime['rating_score'] * 0.2) + (anime['popularity_score'] * 0.2)

In [21]:
# Create feature matrices for cosine similarity
feature_matrices = {
    'genre': tfidf_matrix_genre,
    'rating': anime[['rating_score']],
    'popularity': anime[['popularity_score']],
    'combined': anime[['combined_score']]
}

In [22]:
feature_matrices['genre']

<19311x46 sparse matrix of type '<class 'numpy.float64'>'
	with 61019 stored elements in Compressed Sparse Row format>

In [23]:
# Calculate cosine similarity for each feature
cosine_sims = {}
for feature, matrix in feature_matrices.items():
    cosine_sims[feature] = cosine_similarity(matrix, matrix)

# Combine similarity matrices based on weights
weighted_similarity = (
    cosine_sims['genre'] * 0.6 +
    cosine_sims['rating'] * 0.2 +
    cosine_sims['popularity'] * 0.2
)

In [24]:
# Function to get recommendations based on combined cosine similarity
'''def get_recommendations(title):
    anime_index = anime[anime['title'] == title].index[0]
    sim_scores = list(enumerate(weighted_similarity[anime_index]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]  # Exclude the movie itself, and get top 10 similar movies
    anime_indices = [score[0] for score in sim_scores]
    return anime['title'].iloc[anime_indices]'''

def get_recommendations(title):
    anime_index = anime[anime['title'] == title].index[0]
    print(anime.loc[anime_index])
    sim_scores = list(enumerate(weighted_similarity[anime_index]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Exclude the movie itself and movies with similar titles (same prefix)
    sim_scores = [score for score in sim_scores if not anime['title'].iloc[score[0]].startswith(title)]

    # Keep only the top movie for each title prefix
    unique_titles = set()
    recommended_anime = []
    filtered_scores = []
    for score in sim_scores:
        anime_title = anime['title'].iloc[score[0]]
        title_prefix = ' '.join(anime_title.split()[:-1])  # Exclude the last word (part number)
        if title_prefix not in unique_titles:
            filtered_scores.append(score)
            unique_titles.add(title_prefix)
            #print(anime_title)
            #ind = anime[anime['title'] == anime_title].index[0]
            #print(anime.loc[ind])
            # Append details of the recommended movie
            recommended_anime.append({
                'Title': anime_title,
                'Genre': ', '.join(anime['genre'].iloc[score[0]]),
                'Rating': anime['score'].iloc[score[0]],
                'Popularity': anime['popularity'].iloc[score[0]],
                'members' : anime['members'].iloc[score[0]],
                'Priority Score': score[1]
            })
            if len(recommended_anime)==10:
                break

            # You can include additional details as needed

    # Convert the list of dictionaries to a DataFrame
    recommended_anime_df = pd.DataFrame(recommended_anime)

    return recommended_anime_df


In [25]:
# Example: Get recommendations for a anime
anime_title = 'Death Note'
recommendations = get_recommendations(anime_title)
print(f"Recommended movies for '{anime_title}':")
recommendations.head(10)

uid                                                              1535
title                                                      Death Note
synopsis            A shinigami, as a god of death, can kill any p...
genre               [Mystery, Police, Psychological, Supernatural,...
aired                                     Oct 4, 2006 to Jun 27, 2007
episodes                                                         37.0
members                                                       1871043
popularity                                                          1
ranked                                                           52.0
score                                                            8.65
img_url             https://cdn.myanimelist.net/images/anime/9/945...
link                    https://myanimelist.net/anime/1535/Death_Note
genre_score                                                  3.401159
rating_score                                                 0.937161
popularity_score    

,Title,Genre,Rating,Popularity,members,Priority Score
0,B: The Beginning,"Action, Mystery, Police, Psychological, Supern...",7.510000,721,155265,0.960503
1,B: The Beginning 2,"Action, Mystery, Police, Psychological, Supern...",6.412905,2620,29335,0.960503
2,Mirai Nikki,"Action, Mystery, Psychological, Shounen, Super...",7.740000,13,1205275,0.913182
3,Mousou Dairinin,"Mystery, Dementia, Police, Psychological, Supe...",7.720000,463,226389,0.909007
4,Higurashi no Naku Koro ni Kai,"Mystery, Psychological, Supernatural, Thriller",8.290000,306,317911,0.903055
5,Yakusoku no Neverland 2nd Season,"Sci-Fi, Mystery, Psychological, Thriller, Shounen",6.679565,1124,99979,0.872677
6,Zankyou no Terror,"Mystery, Psychological, Thriller",8.170000,84,642866,0.867316
7,Babylon,"Mystery, Psychological, Thriller",7.770000,1577,64960,0.867316
8,Mi Yu Xing Zhe,"Mystery, Psychological, Thriller",7.530000,4481,9328,0.867316
9,Imawa no Kuni no Alice (OVA),"Action, Psychological, Supernatural, Thriller,...",7.580000,2702,27543,0.862432
